Packages importieren

In [26]:
import pandas as pd
import numpy as np
from numpy.linalg import svd
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
import math
from matplotlib import rcParams
import locale

# === 1. Globale Einstellungen für ALLE Plots ===
rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Latin Modern Sans"],

    # Lade lmodern und sansmath, aktiviere serifenlose Mathematik
    "text.latex.preamble": r"""
        \usepackage{lmodern}
        \usepackage{sansmath}
        \sansmath
        \renewcommand{\familydefault}{\sfdefault}
        \usepackage{icomma}
    """,

     # Größen etc. wie gehabt
    "font.size": 10,              # Standard-Schriftgröße für Paper
    "axes.labelsize": 10,          # Achsentitelgröße
    "axes.titlesize": 12,          # Plottitelgröße
    "legend.fontsize": 9,          # Legendentext
    "xtick.labelsize": 9,          # Tick-Label-Größe X
    "ytick.labelsize": 9,          # Tick-Label-Größe Y

    # Plotgröße und Auflösung
    "figure.figsize": (6, 4),      # etwa 15x10 cm für LaTeX-Spaltenbreite
    "figure.dpi": 300,

    # Ränder
    "savefig.bbox": "tight",
    
    # Linien und Gitter
    "axes.grid": True,             # Gitter standardmäßig an
    "grid.color": "gray",
    "grid.linestyle": "--",
    "grid.linewidth": 0.5,
    
    # Achsenlinien
    "axes.edgecolor": "black",
    "axes.linewidth": 0.8,
    
    # Ticks
    "xtick.direction": "in",       # Ticks nach innen
    "ytick.direction": "in",
    "xtick.major.size": 5,         # Ticklänge
    "ytick.major.size": 5,
    "xtick.minor.size": 2.5,       # Kleine Ticks
    "ytick.minor.size": 2.5,
})

# 1) Deutsche Locale setzen (ggf. liste mit locale -a prüfen)
locale.setlocale(locale.LC_ALL, 'de_DE.UTF-8')

# 2) Matplotlib sagen, die Locale zu benutzen
rcParams['axes.formatter.use_locale'] = True

Funktionen importieren

In [27]:
def kabsch_algorithm(P, Q):
    p_centroid = np.mean(P, axis=0)
    q_centroid = np.mean(Q, axis=0)
    P_centered = P - p_centroid # Werte werden ins Zentrum des Koordinatensystems verschoben
    Q_centered = Q - q_centroid # Werte werden ins Zentrum des Koordinatensystems verschoben
    H = P_centered.T @ Q_centered # Korrelationsmatrix
    U, S, Vt = svd(H)
    V = Vt.T
    R = V @ U.T
    if np.linalg.det(R) < 0:
        V[:,-1] *= -1
        R = V @ U.T
    t = q_centroid - R @ p_centroid
    return R, t

def rotation_matrix_from_euler(alpha, beta, gamma):
    # Rotationsmatrizen für Eulerwinkel (Z-Y-X)
    Rz = np.array([[np.cos(alpha), -np.sin(alpha), 0],
                   [np.sin(alpha),  np.cos(alpha), 0],
                   [0,              0,             1]])
    Ry = np.array([[ np.cos(beta), 0, np.sin(beta)],
                   [0,             1, 0],
                   [-np.sin(beta), 0, np.cos(beta)]])
    Rx = np.array([[1,             0,              0],
                   [0, np.cos(gamma), -np.sin(gamma)],
                   [0, np.sin(gamma),  np.cos(gamma)]])
    return Rz @ Ry @ Rx

def residual(params, P, Q):
    alpha, beta, gamma, tx, ty, tz = params
    R_opt = rotation_matrix_from_euler(alpha, beta, gamma)
    t_opt = np.array([tx, ty, tz])
    P_transformed = (R_opt @ P.T).T + t_opt
    diff = P_transformed - Q
    return diff.flatten()

# Introduce noise into 10 random data points
def add_noise_to_random_points(data, noise_level=0.01, num_points=10):
    indices = np.random.choice(data.shape[0], num_points, replace=False)
    noise = np.random.normal(0, noise_level, (num_points, data.shape[1]))
    data_noisy = data.copy()
    data_noisy[indices] += noise
    return data_noisy


Messdaten aller Messtage 21.01., 01.02., 10.02., 03.03. (Position 1) und 03.03. (Position 2) importieren

In [28]:
# Daten vom 21.01.2025 importieren und strukturieren
data_ts_21_01 = pd.read_csv('../data/Rohdaten_TS_21_01_25.csv', delimiter=';', index_col=0).sort_index()
data_encoder_21_01 = pd.read_csv('../data/Encoderwerte_21_01_25.csv', delimiter=';', index_col=0).sort_index()
validation_data_TS_21_01 = pd.read_csv('../data/Rohdaten_TS_21_01_V_25.csv', delimiter=';', index_col=0).sort_index()
validation_data_encoder_21_01 = pd.read_csv('../data/Encoderwerte_21_01_V_25.csv', delimiter=';', index_col=0).sort_index()

encoder_points_21_01 = (data_encoder_21_01[['x_Encoder', 'y_Encoder', 'z_Encoder']]/1000)
ts_points_21_01 = data_ts_21_01[['x_TS', 'y_TS', 'z_TS']]

encoder_points_validation_21_01 = validation_data_encoder_21_01[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000
ts_points_validation_21_01 = validation_data_TS_21_01[['x_TS', 'y_TS', 'z_TS']].values 

# Daten vom 01.02.2025 importieren und strukturieren
data = pd.read_csv('../data/Modellentwicklung_Encoderwerte.csv', sep=';', index_col=0)
raw_data = pd.read_csv('../data/Modellentwicklung_Rohdaten_Totalstation.csv', delimiter=';', index_col=0)

raw_data.columns = ['x_TS', 'y_TS', 'z_TS']
data_positioning = data[
    (data['Norm'] < 20) & 
    (data['x_Encoder'] < 2000) & 
    (data['x_Encoder'] > -1200) & 
    (data['y_Encoder'] < 8000) & 
    (data['z_Encoder'] > -1000) & 
    (data['z_Encoder'] < 2200)
]
data_positioning.index = data_positioning.index
raw_data_positioning = raw_data[raw_data.index.isin(data_positioning.index)]
ts_points_01_02 = raw_data_positioning[['x_TS','y_TS','z_TS']].sort_index()
encoder_points_01_02 = data_positioning[['x_Encoder','y_Encoder','z_Encoder']].sort_index()/1000
data_encoder_01_02 = data_positioning[['x_Encoder', 'y_Encoder', 'z_Encoder','encoder_0','encoder_1','encoder_2','encoder_3','encoder_4','encoder_5','encoder_6','encoder_7']].sort_index()
used_indices = set(data_positioning.index)
available_indices = list(set(data.index) - used_indices)
selected_indices = np.random.choice(available_indices, 20, replace=False)
validation_data_encoder_01_02 = data.loc[selected_indices].sort_index()
validation_data_TS_01_02 = raw_data.loc[selected_indices].sort_index()
encoder_points_validation_01_02 = validation_data_encoder_01_02[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000 
ts_points_validation_01_02 = validation_data_TS_01_02[['x_TS', 'y_TS', 'z_TS']].values

# Daten vom 10.02.2025 importieren und strukturieren
data = pd.read_csv('../data/Encoderwerte_Rohdaten_10_02_25.csv', delimiter=';',index_col=0)
ts_points_10_02 = data[['x_TS', 'y_TS', 'z_TS']].sort_index()
encoder_points_10_02 = data[['x_Encoder', 'y_Encoder','z_Encoder']].sort_index()
data_encoder_10_02 = data[['x_Encoder', 'y_Encoder','z_Encoder','encoder_0','encoder_1','encoder_2','encoder_3','encoder_4','encoder_5','encoder_6','encoder_7']].sort_index()

# Daten vom 25.02.2025 importieren und strukturieren
data_TS_25_02 = pd.read_csv('../data/Rohdaten_TS_25_02_E_25.csv', delimiter=';', index_col=0).sort_index()
data_encoder_25_02 = pd.read_csv('../data/Encoderwerte_25_02_E_25.csv', delimiter=';', index_col=0).sort_index()
ts_points_25_02 = data_TS_25_02[['x_TS', 'y_TS', 'z_TS']]
encoder_points_25_02 = data_encoder_25_02[['x_Encoder', 'y_Encoder','z_Encoder']]/1000
validation_data_encoder_25_02 = pd.read_csv('../data/Encoderwerte_25_02_25.csv', delimiter=';', index_col=0).sort_index()
validation_data_TS_25_02 = pd.read_csv('../data/Rohdaten_TS_25_02_25.csv', delimiter=';', index_col=0).sort_index()
encoder_points_validation_25_02 = validation_data_encoder_25_02[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000
ts_points_validation_25_02 = validation_data_TS_25_02[['x_TS', 'y_TS', 'z_TS']].values

# Daten vom 03.03.2025 (Position 1) importieren und strukturieren
data_TS_03_03 = pd.read_csv('../data/Rohdaten_TS_03_03_25_Position_1_Einmessung.csv', delimiter=';', index_col=0).sort_index()
data_encoder_03_03 = pd.read_csv('../data/Encoderwerte_03_03_25_Position_1_Einmessung.csv', delimiter=';', index_col=0).sort_index()
ts_points_03_03 = data_TS_03_03[['x_TS', 'y_TS', 'z_TS']]
encoder_points_03_03 = data_encoder_03_03[['x_Encoder', 'y_Encoder','z_Encoder']]/1000
validation_data_encoder_03_03 = pd.read_csv('../data/Encoderwerte_03_03_25_Position_1.csv', delimiter=';', index_col=0).sort_index()
validation_data_TS_03_03 = pd.read_csv('../data/Rohdaten_TS_03_03_25_Position_1.csv', delimiter=';', index_col=0).sort_index()
validation_data_encoder_03_03 = validation_data_encoder_03_03[~validation_data_encoder_03_03.index.str.endswith(("second_stage_corrected", "first_stage_corrected"))].sort_index()
encoder_points_validation_03_03 = validation_data_encoder_03_03[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000
ts_points_validation_03_03 = validation_data_TS_03_03[['x_TS', 'y_TS', 'z_TS']].values

# Daten vom 03.03.2025 (Position 2) importieren und strukturieren
data_TS_03_03_2 = pd.read_csv('../data/Rohdaten_TS_03_03_25_Position_2_Einmessung.csv', delimiter=';', index_col=0).sort_index()
data_encoder_03_03_2 = pd.read_csv('../data/Encoderwerte_03_03_25_Position_2_Einmessung.csv', delimiter=';', index_col=0).sort_index()
ts_points_03_03_2 = data_TS_03_03_2[['x_TS', 'y_TS', 'z_TS']]
encoder_points_03_03_2 = data_encoder_03_03_2[['x_Encoder', 'y_Encoder','z_Encoder']]/1000

validation_data_encoder_03_03_2 = pd.read_csv('../data/Encoderwerte_03_03_25_Position_2.csv', delimiter=';', index_col=0).sort_index()
validation_data_TS_03_03_2 = pd.read_csv('../data/Rohdaten_TS_03_03_25_Position_2.csv', delimiter=';', index_col=0).sort_index()
validation_data_encoder_03_03_2 = validation_data_encoder_03_03_2[~validation_data_encoder_03_03_2.index.str.endswith(("second_stage_corrected", "first_stage_corrected"))].sort_index()
encoder_points_validation_03_03_2 = validation_data_encoder_03_03_2[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000
ts_points_validation_03_03_2 = validation_data_TS_03_03_2[['x_TS', 'y_TS', 'z_TS']].values

Dictionaries für alle TS-Werte, Encoder-Werte und Validierungsdaten für for-Loop erstellen

In [29]:
ts_data = {
    "TS_Points_21_01": ts_points_21_01,
    "TS_Points_01_02": ts_points_01_02,
    "TS_Points_25_02": ts_points_25_02,
    "TS_Points_03_03": ts_points_03_03,
    "TS_Points_03_03_2": ts_points_03_03_2,
}

encoder_data = {
    "Encoder_Points_21_01": encoder_points_21_01,
    "Encoder_Points_01_02": encoder_points_01_02,
    "Encoder_Points_25_02": encoder_points_25_02,
    "Encoder_Points_03_03": encoder_points_03_03,
    "Encoder_Points_03_03_2": encoder_points_03_03_2,
}

validation_ts_data = {
    "TS_Points_validation_21_01": ts_points_validation_21_01,
    "TS_Points_validation_01_02": ts_points_validation_01_02,
    "TS_Points_validation_25_02": ts_points_validation_25_02,
    "TS_Points_validation_03_03": ts_points_validation_03_03,
    "TS_Points_validation_03_03_2": ts_points_validation_03_03_2,
}

validation_encoder_data = {
    "Encoder_Points_validation_21_01": encoder_points_validation_21_01,
    "Encoder_Points_validation_01_02": encoder_points_validation_01_02,
    "Encoder_Points_validation_25_02": encoder_points_validation_25_02,
    "Encoder_Points_validation_03_03": encoder_points_validation_03_03,
    "Encoder_Points_validation_03_03_2": encoder_points_validation_03_03_2,
}

# Convert dictionary keys to lists for ordered processing
ts_keys = list(ts_data.keys())
enc_keys = list(encoder_data.keys())
val_ts_keys = list(validation_ts_data.keys())
val_enc_keys = list(validation_encoder_data.keys())

Auswertungskriterien festlegen

In [30]:
evaluate_sample_size = True
add_noise = False
loss_function = False
min_samples = 4
sample_interval = 6
noise_percent_min = 0
noise_percent_max = 0.1
noise_steps_number = 2
noise_level = 0.1
legend = False
grid_choice = True

Auswertungen dürchführen

In [31]:
df_result = pd.DataFrame(index=None)
# Iterate while there are elements in all dictionaries
while ts_keys and enc_keys and val_ts_keys and val_enc_keys:
    # Get the first keys from each dictionary
    ts_key = ts_keys.pop(0)
    enc_key = enc_keys.pop(0)
    val_ts_key = val_ts_keys.pop(0)
    val_enc_key = val_enc_keys.pop(0)
    
    # Extract the first series from each dictionary
    ts_points = ts_data.pop(ts_key)  # Remove and get the time series data
    encoder_points = encoder_data.pop(enc_key)  # Remove and get the encoder data
    ts_points_validation = validation_ts_data.pop(val_ts_key)  # Remove and get the validation data
    encoder_points_validation = validation_encoder_data.pop(val_enc_key)  # Remove and get the validation data
    
    if evaluate_sample_size:
        for num_points_data in range(min_samples, len(ts_points), sample_interval):
            # Zufällige Punkte auswählen
            indices = np.random.choice(ts_points.index, num_points_data, replace=False)

            random_ts_points = ts_points.loc[indices]
            random_encoder_points = encoder_points.loc[indices]

            df_result.loc[len(df_result),'sample_size'] = num_points_data
                    # Ensure they have the same label
            assert all(random_ts_points.index == random_encoder_points.index), "Indices do not match!"

            P = random_ts_points.values
            Q = random_encoder_points.values
            if add_noise:
                percentages = np.linspace(noise_percent_min, noise_percent_max, noise_steps_number)
            else:
                percentages = [0]
            for percent in percentages:
                if percent > 0:
                    P_noisy = add_noise_to_random_points(P, noise_level, num_points= math.ceil(num_points_data*percent))

                    #Check noise
                    noise_check = np.linalg.norm(P_noisy - P, axis=1)*1000
                    P = P_noisy
                
                # ------------------------------
                # Erster Schritt: Kabsch-Algorithmus zur Initialschätzung
                # ------------------------------
                R_est, t_est = kabsch_algorithm(P, Q)

                # Wir gehen davon aus, dass R_est bereits sehr nahe ist. 
                # Für die robuste Optimierung parametrisieren wir R über Eulerwinkel.
                # Hier vereinfachen wir und starten mit Eulerwinkeln nahe Null.
                x0 = np.array([0.0, 0.0, 0.0, t_est[0], t_est[1], t_est[2]])
                
                if loss_function:
                    methods = ['huber', 'soft_l1', 'cauchy', 'arctan', 'linear']
                else:
                    methods = ['huber']
                for method_loss in methods:
                    # ------------------------------
                    # Robuste Optimierung mit Huber-Loss
                    # ------------------------------
                    result = least_squares(residual, x0, args=(P, Q), loss= method_loss, f_scale=1.0)
                    alpha_opt, beta_opt, gamma_opt, tx_opt, ty_opt, tz_opt = result.x
                    residuals_after_kabsch_local = residual(result.x, P, Q)
                    R_opt = rotation_matrix_from_euler(alpha_opt, beta_opt, gamma_opt)
                    t_opt = np.array([tx_opt, ty_opt, tz_opt])

                    # Berechnen Sie den durchschnittlichen Fehler nach der Optimierung
                    err_final = np.linalg.norm((R_opt @ P.T).T + t_opt - Q, axis=1)
                    err_final_mean = np.mean(err_final)
                    date_key = ts_key.replace('TS_Points_', '')
                    # Properly append the results as a new row
                    label_col = 'error_validation_' + str(percent)  + '_' + method_loss + '_' + date_key
                    # ------------------------------
                    # Validation
                    # ------------------------------
                    ts_points_validation_transformed = (R_opt @ ts_points_validation.T).T + t_opt

                    err_final_validation = np.linalg.norm(ts_points_validation_transformed - encoder_points_validation, axis=1)
                    df_result.loc[len(df_result)-1, label_col] = np.mean(err_final_validation)*1000

for col in df_result.columns:
    if col == 'sample_size':
        continue
    date_dataset = col.split('_')[-3] + '_' + col.split('_')[-2] + '_' + col.split('_')[-1]
    date_key = '_' + date_dataset.replace('huber_', '')
    plt.scatter(df_result['sample_size'],df_result[col], label= col, marker='x')
    plt.xlabel('Anzahl der Stichproben')
    plt.ylabel('Mittlerer Fehler der Norm in mm')
    if legend:
        plt.legend()

    plt.savefig('../results/Diagramme/Einmessverfahren/Einfluss_Stichprobenanzahl_Einmessung' + date_key + '.pdf')
    plt.close()